In [5]:
# import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from mlxtend.frequent_patterns import apriori, association_rules
from imblearn.over_sampling import SMOTE

In [4]:
# load dataframes
spend_revenue = pd.read_csv(r"/Users/adityamxr/Desktop/finding-marketing-insights/data-analysis/spend_revenue.csv")
sales_full_merged = pd.read_csv(r"/Users/adityamxr/Desktop/finding-marketing-insights/data-analysis/sales_full_merged.csv")
rfm = pd.read_csv(r"/Users/adityamxr/Desktop/finding-marketing-insights/models/rfm.csv")

In [6]:
sales_full_merged.head()

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,Month,Coupon_Code,Discount_pct,GST,Invoice_Value,YearMonth,First_Purchase_Month,YearWeek,DayOfWeek
0,17850,16679,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,1,ELEC10,10.0,0.10,144.977339,2019-01,2019-01,2018-12-31/2019-01-06,Tuesday
1,17850,16680,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,1,ELEC10,10.0,0.10,144.977339,2019-01,2019-01,2018-12-31/2019-01-06,Tuesday
2,17850,16681,2019-01-01,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05,6.5,Used,1,OFF10,10.0,0.10,8.346845,2019-01,2019-01,2018-12-31/2019-01-06,Tuesday
3,17850,16682,2019-01-01,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53,6.5,Not Used,1,SALE10,10.0,0.18,85.526993,2019-01,2019-01,2018-12-31/2019-01-06,Tuesday
4,17850,16682,2019-01-01,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50,6.5,Used,1,AIO10,10.0,0.18,21.376730,2019-01,2019-01,2018-12-31/2019-01-06,Tuesday


In [8]:
# data prep
basket_prd_cat = sales_full_merged.groupby(['Transaction_ID', 'Product_Category'])['Quantity'].sum().unstack().reset_index().fillna(0)

# convert all positive quantities to 1 (indicating the product was bought)
basket_prd_cat = basket_prd_cat.set_index('Transaction_ID')
basket_prd_cat = basket_prd_cat.applymap(lambda x: 1 if x > 0 else 0)

# display the basket (transaction format)
basket_prd_cat.head()

Product_Category,Accessories,Android,Apparel,Backpacks,Bags,Bottles,Drinkware,Fun,Gift Cards,Google,Headgear,Housewares,Lifestyle,More Bags,Nest,Nest-Canada,Nest-USA,Notebooks & Journals,Office,Waze
Transaction_ID,,,,,,,,,,,,,,,,,,,,
16679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
16680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
16681,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16682,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
16684,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [9]:
# apply apropi algorithm to identify frequent combinations of product categories that are often bought together

# apply the apriori algorithm with a minimum support threshold (0.01 for 1% of transactions)
frequent_itemsets_prd_cat = apriori(basket_prd_cat, min_support=0.01, use_colnames=True)

# view the frequent itemsets
frequent_itemsets_prd_cat.head()

/Users/adityamxr/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.324369,(Apparel)
1,0.061650,(Bags)
2,0.010295,(Bottles)
3,0.100714,(Drinkware)
4,0.026894,(Headgear)


In [11]:
# generate association rules
rules_prd_cat = association_rules(frequent_itemsets_prd_cat, metric="lift", min_threshold=1.0)

# view the association rules
rules_prd_cat[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
0,(Bags),(Apparel),0.025657,0.416181,1.283051
1,(Apparel),(Bags),0.025657,0.079100,1.283051
2,(Apparel),(Drinkware),0.045010,0.138762,1.377784
3,(Drinkware),(Apparel),0.045010,0.446910,1.377784
4,(Headgear),(Apparel),0.017318,0.643917,1.985140
...,...,...,...,...,...
69,"(Office, Lifestyle)",(Drinkware),0.016719,0.476136,4.727596
70,"(Drinkware, Lifestyle)",(Office),0.016719,0.646605,4.595736
71,(Office),"(Drinkware, Lifestyle)",0.016719,0.118832,4.595736
72,(Drinkware),"(Office, Lifestyle)",0.016719,0.166006,4.727596


## Market Basket Analysis Summary:


The market basket analysis identified significant product associations, such as **Apparel being purchased with Drinkware (support = 4.50%, lift = 1.38)** and **Bags with Apparel (support = 2.57%, lift = 1.28)**. The association between **Headgear and Apparel has a lift of 1.98, suggesting a high potential for cross-selling these categories**. High-lift rules, such as **Office and Lifestyle with Drinkware (lift = 4.72), present strong opportunities for targeted promotions**. These insights allow stakeholders to create bundling offers, for example, promoting Apparel with Drinkware or providing discounts when customers buy both Office and Lifestyle items. By leveraging these high-confidence and high-lift associations, personalized recommendations or promotions could boost overall basket size and drive increased revenue. Stakeholders could also use this analysis to optimize inventory management by ensuring that frequently co-purchased items are stocked together in marketing campaigns or physical store layouts.